# FAISS + Confidence-Based Reranking for SKU Matching


This notebook implements a two-step retrieval pipeline using FAISS (top-20 retrieval) followed by confidence-based reranking to evaluate accuracy and false positives across confidence thresholds.


## 1. Load Input Dataset

In [ ]:
# 0. Load data
csv_file = '/content/drive/MyDrive/HackWeekC1/HackWeekProductsData.csv'
df_source = pd.read_csv(csv_file, encoding='utf-8-sig')
print("Number of rows:", len(df_source))

Number of rows: 4165840


In [ ]:
# 1) Count how many examples per SKU
sku_counts = df_source['ProductMasterId'].value_counts()

In [ ]:
# 2) Print basic stats
print(f"Total distinct SKUs: {sku_counts.shape[0]}")
print(f"  • Max occurrences: {sku_counts.max()}  (SKU {sku_counts.idxmax()})")
print(f"  • Min occurrences: {sku_counts.min()}  (example SKUs: {sku_counts[sku_counts==sku_counts.min()].index[:5].tolist()}…)")


Total distinct SKUs: 95016
  • Max occurrences: 1168385  (SKU 0)
  • Min occurrences: 1  (example SKUs: [90449, 130728, 20962, 122172, 145046]…)


## 2. Count Examples Per SKU
Count how many times each SKU appears to understand class distribution.

In [ ]:
# 4) See the long tail:
print("\nNumber of SKUs with ≤5 examples:", (sku_counts <= 5).sum())
print("Number of SKUs with ≥5 examples:", (sku_counts >= 5).sum())
print("Number of SKUs with ≥10 examples:", (sku_counts >= 10).sum())
print("Number of SKUs with ≥20 examples:", (sku_counts >= 20).sum())
print("Number of SKUs with ≥50 examples:", (sku_counts >= 50).sum())
print("Number of SKUs with ≥100 examples:", (sku_counts >= 100).sum())
print("Number of SKUs with ≥1000 examples:", (sku_counts >= 1000).sum())


Number of SKUs with ≤5 examples: 50664
Number of SKUs with ≥5 examples: 48507
Number of SKUs with ≥10 examples: 34473
Number of SKUs with ≥20 examples: 23350
Number of SKUs with ≥50 examples: 12253
Number of SKUs with ≥100 examples: 6760
Number of SKUs with ≥1000 examples: 168


## 3. Train FAISS + Reranker Model
Run FAISS-based retrieval on queries, then apply a logistic reranker to compute top-1 match confidence scores and evaluate performance.


In [ ]:
# challenge1_faiss_rerank_top5_sampling.py
# Two-stage retrieval + rerank pipeline with top-5 evaluation
# Sampling: fixed examples per SKU + overall max rows
# Usage: Run in Google Colab. Upload your CSV (e.g., 'HackWeekProductsData.csv') to working directory.

# 0. Install dependencies
!pip install --quiet faiss-cpu sentence-transformers scikit-learn pandas numpy rapidfuzz

# 1. Imports
import pandas as pd
import numpy as np
import re
import faiss
from sentence_transformers import SentenceTransformer
from rapidfuzz import fuzz
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 2. Load full data then sample by SKU
csv_file = '/content/drive/MyDrive/HackWeekC1/HackWeekProductsData.csv'
# sampling params
min_examples = 50     # examples per SKU
max_rows     = 600000  # total rows cap
print(f"Loading full data from {csv_file}...")
df_full = pd.read_csv(csv_file, encoding='utf-8-sig')
print(f"→ Total rows read: {len(df_full)}")

# 3. Filter to 'Map' rows
if 'MappingModeName' in df_full.columns:
    before = len(df_full)
    df_full = df_full[df_full['MappingModeName'].str.strip().str.lower()=='map']
    print(f"Filtered to {len(df_full)} mapped rows (dropped {before-len(df_full)}).")
else:
    print("No MappingModeName; proceeding with all rows.")

# 4. Select SKUs with >= min_examples occurrences
sku_counts = df_full['ProductMasterId'].value_counts()
eligible_skus = sku_counts[sku_counts >= min_examples].index
print(f"Found {len(eligible_skus)} SKUs with >= {min_examples} examples.")

# 5. Limit number of SKUs by total cap
max_skus = max_rows // min_examples
selected_skus = list(sku_counts.loc[eligible_skus].nlargest(max_skus).index)
print(f"Selecting top {len(selected_skus)} SKUs (max cap {max_skus}) => target rows {len(selected_skus)*min_examples}")

# 6. Sample min_examples per SKU
df_source = (
    df_full[df_full['ProductMasterId'].isin(selected_skus)]
    .groupby('ProductMasterId', group_keys=False)
    .apply(lambda g: g.sample(n=min_examples, random_state=42))
    .reset_index(drop=True)
)
print(f"Sampled dataframe: {len(df_source)} rows across {df_source['ProductMasterId'].nunique()} SKUs.")

# 7. Prepare master DataFrame—only selected SKUs
source_cols = [
    'ServiceAndProductMappingId','SourceMasterBrand','SourceBrand','SourceDescription',
    'SourceCategory','SourceSubcategory','SourceSize','SourceUnitOfMeasure',
    'CleanBarcode','IsValidBarcode','ProductMasterId'
]
master_cols = [
    'ProductMasterId','MasterBrandName','BrandName','SubBrandName',
    'ProductName','CategoryName','SubcategoryFormName','SubcategoryFunctionName',
    'ProductSizeOnLabel','CleanBarcode'
]
missing = set(source_cols+master_cols) - set(df_full.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")
# filter master to only selected SKUs

df_master = (
    df_full[df_full['ProductMasterId'].isin(selected_skus)]
    [master_cols]
    .drop_duplicates('ProductMasterId')
    .set_index('ProductMasterId')
)
print(f"Master SKUs in index: {len(df_master)}")

# 8. Build KB texts
kb_ids = list(df_master.index)
kb_texts = [
    ' '.join(filter(None, [row.MasterBrandName, row.BrandName, row.ProductName,
                            row.CategoryName, row.SubcategoryFormName, row.SubcategoryFunctionName,
                            str(row.ProductSizeOnLabel)]))
    for _, row in df_master.iterrows()
]

# 9. Normalize queries
def normalize_source(r):
    parts = [r.SourceMasterBrand, r.SourceBrand, r.SourceDescription,
             r.SourceCategory, r.SourceSubcategory, str(r.SourceSize), str(r.SourceUnitOfMeasure)]
    return ' '.join(str(p) for p in parts if pd.notna(p) and p)

df_source['query_text'] = df_source.apply(normalize_source, axis=1)

# 10. Embed
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("Embedding KB...")
kb_emb = embedder.encode(kb_texts, show_progress_bar=True)
print("Embedding queries...")
query_emb = embedder.encode(df_source['query_text'].tolist(), show_progress_bar=True)

# 11. FAISS index
faiss.normalize_L2(kb_emb)
dim = kb_emb.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(kb_emb)
print(f"FAISS index built: {index.ntotal} entries, dim={dim}")

# 12. Retrieve top-K
faiss.normalize_L2(query_emb)
k = 20
D, I = index.search(query_emb, k)
print(f"Retrieved top-{k} candidates for {len(df_source)} queries.")

# 13. Parse size feature
def parse_size(text):
    try:
        val, unit = re.search(r"(\d+\.?\d*)\s*(ml|l|oz|fl oz)", str(text).lower()).groups()
        unit_map = {'ml':1,'l':1000,'oz':29.5735,'fl oz':29.5735}
        return float(val) * unit_map[unit]
    except:
        return np.nan

df_source['source_ml'] = df_source['SourceSize'].apply(parse_size)
df_master['master_ml'] = df_master['ProductSizeOnLabel'].apply(parse_size)

# 14. Unique barcode lookup
df_valid = df_source[df_source.IsValidBarcode==1]
bc_counts = df_valid.groupby('CleanBarcode')['ProductMasterId'].nunique()
unique_bcs = set(bc_counts[bc_counts==1].index)
barcode_dict = {bc:pid for pid,bc in zip(df_master.index, df_master['CleanBarcode']) if bc in unique_bcs}

# 15. Split data
dev, test = train_test_split(df_source, test_size=0.2, random_state=42, stratify=df_source['ProductMasterId'])
print(f"Split into Train: {len(dev)}, Test: {len(test)} rows.")

# 16. Feature extractor
def make_feature(src, pid):
    m = df_master.loc[pid]
    feats = []
    feats.append(int(src.IsValidBarcode==1 and src.CleanBarcode==m.CleanBarcode))
    feats.append(int(src.SourceCategory==m.CategoryName))
    s, mm = src.source_ml, m.master_ml
    feats.append((min(s,mm)/max(s,mm)) if pd.notna(s) and pd.notna(mm) else 0)
    feats.append(fuzz.token_set_ratio(src.query_text, ' '.join([m.BrandName, m.ProductName]))/100)
    return feats

# 17. Train reranker
t_pairs = []
for _, src in dev.iterrows():
    t_pairs.append((src, src.ProductMasterId, 1))
    cnt = 0
    for j in I[src.name][:5]:
        pid = kb_ids[j]
        if pid != src.ProductMasterId:
            t_pairs.append((src, pid, 0)); cnt += 1
            if cnt >= 3: break
X_train = np.array([make_feature(s,p) for s,p,_ in t_pairs])
y_train = np.array([l for _,_,l in t_pairs])
clf = CalibratedClassifierCV(LogisticRegression(max_iter=1000), cv=3)
clf.fit(X_train, y_train)
print("Reranker trained.")

# 18. Predict & evaluate
results = []
for i, src in test.iterrows():
    if src.IsValidBarcode==1 and src.CleanBarcode in barcode_dict:
        ids, confs = [barcode_dict[src.CleanBarcode]], [1.0]
    else:
        ids, confs = [], []
        for j in I[i][:5]:
            pid = kb_ids[j]
            prob = clf.predict_proba([make_feature(src, pid)])[0,1]
            ids.append(pid); confs.append(prob)
    actual = src.ProductMasterId
    is_correct = actual in ids
    pred_str = '|'.join(map(str, ids))
    conf_str = '|'.join(f"{c:.2f}" for c in confs)
    results.append((src.ServiceAndProductMappingId, pred_str, conf_str, actual, is_correct))

res_df = pd.DataFrame(results, columns=['ID','Pred','Conf','Actual','Correct'])

# 19. Metrics and export
overall_hit = res_df['Correct'].mean()
single = ~res_df['Pred'].str.contains('|', regex=False)
acc_single = res_df[single]['Correct'].mean()
acc_multi = res_df[~single]['Correct'].mean()
print(f"Accuracy@1 (single-ID cases): {acc_single:.2%}")
print(f"Hit@5 (multi-ID cases): {acc_multi:.2%}")
print(f"Overall Hit@5: {overall_hit:.2%}")

import csv
# point this at your mounted Drive folder
out_folder = '/content/drive/MyDrive/HackWeekC1/'

# build the filename with your metrics
filename = out_folder + f"faiss_top5_results_{overall_hit*100:.2f}pct_{len(res_df)}rows.csv"

# export (ensure you still have `import csv` at the top)
res_df.to_csv(filename, index=False, quoting=csv.QUOTE_ALL)
print(f"Results saved to: {filename}")


Loading full data from /content/drive/MyDrive/HackWeekC1/HackWeekProductsData.csv...
→ Total rows read: 4165840
Filtered to 2996970 mapped rows (dropped 1168870).
Found 12249 SKUs with >= 50 examples.
Selecting top 12000 SKUs (max cap 12000) => target rows 600000


/tmp/ipython-input-819996495.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=min_examples, random_state=42))


Sampled dataframe: 600000 rows across 12000 SKUs.
Master SKUs in index: 12000
Embedding KB...


Batches:   0%|          | 0/375 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Embedding queries...


Batches:   0%|          | 0/18750 [00:00<?, ?it/s]

FAISS index built: 12000 entries, dim=384
Retrieved top-20 candidates for 600000 queries.
Split into Train: 480000, Test: 120000 rows.
Reranker trained.
Accuracy@1 (single-ID cases): 99.81%
Hit@5 (multi-ID cases): 48.17%
Overall Hit@5: 55.09%
Results saved to: /content/drive/MyDrive/HackWeekC1/faiss_top5_results_55.09pct_120000rows.csv


## 4. Identify Mapped SKUs
Reload the full dataset to see what percentage of all SKUs are covered by our training subset.


In [ ]:
import pandas as pd

# 1. Load full CSV (mapped rows only)
path = '/content/drive/MyDrive/HackWeekC1/HackWeekProductsData.csv'
df = pd.read_csv(path, encoding='utf-8-sig')
if 'MappingModeName' in df.columns:
    df = df[df['MappingModeName'].str.strip().str.lower()=='map']

# 2. Count SKUs and total mapped rows
total_mapped = len(df)
sku_counts = df['ProductMasterId'].value_counts()

# 3. Identify SKUs with ≥50 examples
skus_50 = sku_counts[ sku_counts >= 50 ].index
rows_50 = df[df['ProductMasterId'].isin(skus_50)].shape[0]

print(f"Total mapped rows: {total_mapped}")
print(f"Distinct SKUs overall: {len(sku_counts)}")
print(f"SKUs with ≥50 examples: {len(skus_50)}")
print(f"Rows for SKUs ≥50 examples: {rows_50} ({rows_50/total_mapped:.2%} of mapped)")


Total mapped rows: 2996970
Distinct SKUs overall: 95010
SKUs with ≥50 examples: 12249
Rows for SKUs ≥50 examples: 2312153 (77.15% of mapped)


## 5. Load Test Results and Analyze False Positives
Analyze false positive counts at different confidence thresholds (≥0.00 to ≥0.95) and observe their trends.


In [ ]:
import pandas as pd
import csv

# 1. Load your Top-5 results CSV
results_path = '/content/drive/MyDrive/HackWeekC1/faiss_top5_results_55.09pct_120000rows.csv'
res = pd.read_csv(results_path, quoting=csv.QUOTE_ALL)

# 2. Keep only the multi-ID (pipe-separated) cases
mask_multi = res['Pred'].str.contains('|', regex=False)
df_multi = res[mask_multi].copy()

# 3. Extract max confidence from each pipe-separated string
df_multi['max_conf'] = (
    df_multi['Conf']
      .str.split('|')
      .apply(lambda parts: max(float(p) for p in parts))
)

# 4. Overall false positives
total_multi = len(df_multi)
false_total = df_multi[~df_multi['Correct']].shape[0]
print(f"Multi-ID rows: {total_multi}, False-positives: {false_total}")

# 5. False-positive counts & rates at thresholds 0.50, 0.60, 0.70, 0.90
for thr in [0.00,0.10,0.20,0.30,0.40,0.50, 0.60, 0.70, 0.80, 0.90, 0.95]:
    sub = df_multi[df_multi['max_conf'] >= thr]
    fp = sub[~sub['Correct']].shape[0]
    print(f"Threshold ≥{thr:.2f}: {len(sub)} rows → {fp} false-positives ({fp/len(sub):.2%})")


Multi-ID rows: 103907, False-positives: 53859
Threshold ≥0.00: 103907 rows → 53859 false-positives (51.83%)
Threshold ≥0.10: 103907 rows → 53859 false-positives (51.83%)
Threshold ≥0.20: 49548 rows → 21892 false-positives (44.18%)
Threshold ≥0.30: 18439 rows → 3202 false-positives (17.37%)
Threshold ≥0.40: 10537 rows → 935 false-positives (8.87%)
Threshold ≥0.50: 9916 rows → 791 false-positives (7.98%)
Threshold ≥0.60: 9758 rows → 781 false-positives (8.00%)
Threshold ≥0.70: 9758 rows → 781 false-positives (8.00%)
Threshold ≥0.80: 9758 rows → 781 false-positives (8.00%)
Threshold ≥0.90: 8839 rows → 686 false-positives (7.76%)
Threshold ≥0.95: 2523 rows → 170 false-positives (6.74%)


## 6. Compute Accuracy at Thresholds (Excluding Conf = 1.0)
Load results and compute overall accuracy at different thresholds (ignoring rows with confidence == 1.0).


In [ ]:
import pandas as pd
import csv

# 1) Load your saved Top-5 results CSV
res_df = pd.read_csv(
    '/content/drive/MyDrive/HackWeekC1/faiss_top5_results_55.09pct_120000rows.csv',
    quoting=csv.QUOTE_ALL
)

print("Columns in results:", res_df.columns.tolist())
# Make sure you see a column named exactly 'Correct'.

# 2) Split into single-ID vs multi-ID
single_mask = ~res_df['Pred'].str.contains('|', regex=False)
multi_mask  = ~single_mask

# 3) For multi-ID, compute max_conf across the pipe-separated Conf string
res_df.loc[multi_mask, 'max_conf'] = (
    res_df.loc[multi_mask, 'Conf']
          .str.split('|')
          .apply(lambda parts: max(float(p) for p in parts))
)

# 4) Now loop thresholds including 1.0
for thr in [1.0, 0.95, 0.90, 0.80, 0.70, 0.60, 0.50]:
    if thr == 1.0:
        sub = res_df[single_mask]
    else:
        sub = res_df[multi_mask & (res_df['max_conf'] >= thr)]
    n  = len(sub)
    fp = sub[~sub['Correct']].shape[0]
    acc = sub['Correct'].mean() if n else float('nan')
    print(f"Thr ≥{thr:.2f}: {n:6d} rows → {fp:4d} false-positives,  Accuracy: {acc:.2%}")


Columns in results: ['ID', 'Pred', 'Conf', 'Actual', 'Correct']
Thr ≥1.00:  16093 rows →   30 false-positives,  Accuracy: 99.81%
Thr ≥0.95:   2523 rows →  170 false-positives,  Accuracy: 93.26%
Thr ≥0.90:   8839 rows →  686 false-positives,  Accuracy: 92.24%
Thr ≥0.80:   9758 rows →  781 false-positives,  Accuracy: 92.00%
Thr ≥0.70:   9758 rows →  781 false-positives,  Accuracy: 92.00%
Thr ≥0.60:   9758 rows →  781 false-positives,  Accuracy: 92.00%
Thr ≥0.50:   9916 rows →  791 false-positives,  Accuracy: 92.02%



## 7. Compute Accuracy Including Confidence == 1.0
Update analysis to include rows with Conf = 1.0 to get complete accuracy picture.

In [ ]:
import pandas as pd
import csv

# 1) Load your Top-5 results CSV from Drive
results_path = '/content/drive/MyDrive/HackWeekC1/faiss_top5_results_55.09pct_120000rows.csv'
res_df = pd.read_csv(results_path, quoting=csv.QUOTE_ALL, encoding='utf-8-sig')

# 2) Inspect what columns you have
print("Columns in results:", res_df.columns.tolist())

# Expect at least: ['ID','Pred','Conf','Actual','Correct']

# 3) Compute a unified best_conf per row (works for single or pipe-separated values)
def get_best_conf(cell):
    parts = str(cell).split('|')       # even "1.00" → ["1.00"]
    return max(float(p) for p in parts)

res_df['best_conf'] = res_df['Conf'].apply(get_best_conf)

# 4) Evaluate false-positives & accuracy at multiple thresholds
thresholds = [1.00, 0.95, 0.90, 0.80, 0.70, 0.60, 0.50,0.40,0.30,0.20,0.10,0.05,0.00]
for thr in thresholds:
    sub = res_df[ res_df['best_conf'] >= thr ]
    n  = len(sub)
    fp = sub[~sub['Correct']].shape[0]
    acc = sub['Correct'].mean() if n>0 else float('nan')
    print(f"Threshold ≥{thr:.2f}: {n:6d} rows → {fp:4d} false-positives,  Accuracy: {acc:.2%}")


Columns in results: ['ID', 'Pred', 'Conf', 'Actual', 'Correct']
Threshold ≥1.00:  16093 rows →   30 false-positives,  Accuracy: 99.81%
Threshold ≥0.95:  18616 rows →  200 false-positives,  Accuracy: 98.93%
Threshold ≥0.90:  24932 rows →  716 false-positives,  Accuracy: 97.13%
Threshold ≥0.80:  25851 rows →  811 false-positives,  Accuracy: 96.86%
Threshold ≥0.70:  25851 rows →  811 false-positives,  Accuracy: 96.86%
Threshold ≥0.60:  25851 rows →  811 false-positives,  Accuracy: 96.86%
Threshold ≥0.50:  26009 rows →  821 false-positives,  Accuracy: 96.84%
Threshold ≥0.40:  26630 rows →  965 false-positives,  Accuracy: 96.38%
Threshold ≥0.30:  34532 rows → 3232 false-positives,  Accuracy: 90.64%
Threshold ≥0.20:  65641 rows → 21922 false-positives,  Accuracy: 66.60%
Threshold ≥0.10: 120000 rows → 53889 false-positives,  Accuracy: 55.09%
Threshold ≥0.05: 120000 rows → 53889 false-positives,  Accuracy: 55.09%
Threshold ≥0.00: 120000 rows → 53889 false-positives,  Accuracy: 55.09%
